In [1]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

g = readgraph(:condmat2003);
a = adjacency_list(g::AGraph);

N = size(a)[1];

A0 = sparse(1:N,1:N,0*vec(ones(Int64,N,1)),N,N);

for i=1:N
    l = a[i];
    for j=1:length(l)
        A0[i,l[j]]=1;
        A0[l[j],i]=1;
    end
end
A,p = largest_component(A0);
N = size(A,1);
A0=0;p=0;a=0;g=0;
gc()

In [2]:
d = sum(A,2);
s = sum(d);

r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[36.4986, 33.6375, 30.051, 28.4958, 27.987, 27.6143, 25.5505, 25.2162, 24.7182, 23.9871, 23.7489, 23.5479, 22.9412, 22.619, 22.2632, 22.0368, 21.4468, 21.3498, 20.8, 20.7077, 20.572, 20.3278, 20.0171, 20.074, 19.7409, 19.8613, 19.4958, 19.483, 19.4735, 19.3335, 19.2744, 19.0672, 18.8582, 18.9711, 18.8189, 18.8249, 18.4676, 18.3748, 18.2713, 18.2837, 18.2391, 17.7983, 17.7979, 17.8315, 17.6115, 17.6913, 17.5027, 17.5218, 17.4451, 17.4422, 17.1246, 16.964, 17.0284, 16.8084, 16.8395, 16.6252, 16.4345, 16.3742, 16.3724, 16.2507, 16.1111, 16.0253, 15.9379, 15.9756, 15.8838, 15.891, 15.7979, 15.5909, 15.5475, 15.5596, 15.5021, 15.4094, 15.4181, 15.2835, 15.2816, 15.2608, 15.1837, 15.146, 15.1258, 15.0701, 14.9879, 14.9847, 14.88, 14.8637, 14.8376, 14.8401, 14.7027, 14.6899, 14.6558, 14.6098, 14.6585, 14.4042, 14.4053, 14.4445, 14.4055, 14.3492, 14.3538, 14.2381, 14.2753, 14.2075]

In [22]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:80]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 1
Number of communities: 64
Modularity: 0.613025155607062


In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:100
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


LoadError: [91mInterruptException:[39m